# Calculating Mean Average Precision from yolo format txt files

In [ ]:
import glob,os
import numpy as np
import supervision as sv
import pandas as pd
import warnings
warnings.filterwarnings('ignore', category = FutureWarning)
pd.set_option('mode.chained_assignment', None)

In [ ]:
def box_iou_batch(
	boxes_a: np.ndarray, boxes_b: np.ndarray
) -> np.ndarray:

    def box_area(box):
        return (box[2] - box[0]) * (box[3] - box[1])

    area_a = box_area(boxes_a.T)
    area_b = box_area(boxes_b.T)

    top_left = np.maximum(boxes_a[:, None, :2], boxes_b[:, :2])
    bottom_right = np.minimum(boxes_a[:, None, 2:], boxes_b[:, 2:])

    area_inter = np.prod(
    	np.clip(bottom_right - top_left, a_min=0, a_max=None), 2)
        
    return area_inter / (area_a[:, None] + area_b - area_inter)


def non_max_suppression(
   predictions: np.ndarray, iou_threshold: float = 0.5
) -> np.ndarray:
    rows, columns = predictions.shape
    sort_index = np.flip(predictions[:, 5].argsort())
    predictions = predictions[sort_index]
    boxes = predictions[:, :4]
    categories = predictions[:, 4]
    ious = box_iou_batch(boxes, boxes)
    ious = ious - np.eye(rows)

    keep = np.ones(rows, dtype=bool)

    for index, (iou, category) in enumerate(zip(ious, categories)):
        if not keep[index]:
            continue

        condition = (iou > iou_threshold) & (categories == category)
        keep = keep & ~condition

    return keep[sort_index.argsort()]



def xyxy(arr, type = None):
    # arr -- numpy array (n x 6)
    tmp = []
    for x in arr:
        actual_w = x[3] * 1120
        actual_h = x[4] * 1120
        actual_x_c = x[1] * 1120
        actual_y_c = x[2] * 1120
        xmin , ymin =  actual_x_c - (actual_w/2), actual_y_c - (actual_h/2)
        xmax, ymax = actual_x_c + (actual_w/2), actual_y_c + (actual_h/2)
        if type == 'P':
            tmp.append([xmin, ymin, xmax, ymax, x[0], x[-1]]) # xmin, xmax, ymin,ymax, class, conf
        else:
            tmp.append([xmin, ymin, xmax, ymax, x[0]])

    return np.array(tmp)


df = pd.DataFrame(columns = ['map50','map75','map50-95'], index = ['overall', 'FCBK', 'Zigzag'])    

In [ ]:
a = []
nms = False
num_folds = 4
for fold in range(num_folds):
    gt_dir =  f'/user/delhi_sarath_grid_aa_v1/{fold}/test/labels'
    pred_dir = f'/user/Fast_RCNN/crossval/high_res_v1_Delhi/{fold}'

    gt_files = glob.glob(os.path.join(gt_dir,'*.txt'))
    pred_files = glob.glob(os.path.join(pred_dir,'*.txt'))

    gt_results = []
    pred_files = []
    pred_results = []
    info = False

    for i in range(len(gt_files)):
        # gt_results.append(xyxy(np.loadtxt(gt_files[i], ndmin= 2)))
        file_name = os.path.basename(gt_files[i])
        corresponding_pred_file = os.path.join(pred_dir, file_name)
        if os.path.exists(corresponding_pred_file):
            pred_files.append(corresponding_pred_file)
            gt_results.append(xyxy(np.loadtxt(gt_files[i], ndmin= 2)))

    print(pred_files)
    if nms:
        for i in range(len(pred_files)):
            results = np.loadtxt(pred_files[i], ndmin= 2) # load all predictions of single image
            bool_list = non_max_suppression(xyxy(results, type = 'P')) #apply non max suppression
            best_boxes = []
            for i, element in enumerate(bool_list):
                if element:
                    best_boxes.append(results[i]) # add bbox with highest IOU and highest score
            pred_results.append(xyxy(best_boxes, type = 'P'))
            

    else:
        for i in range(len(pred_files)):
            if pred_files[i] == None:
                continue
            pred_results.append(xyxy(np.loadtxt(pred_files[i], ndmin= 2), type='P'))


    if info:
        print(f'No of GT images in Fold {fold}:', len(gt_results))
        print(f'No of bbox in one GT image in Fold {fold}', len(gt_results[0]), type(gt_results[0]))
        print('No of Pred images:', len(pred_results))
        print('No of predictions in one GT image', len(pred_results[0]), type(pred_results[0]))
        print('Shape of predictions in one GT image',pred_results[0].shape)

    #Finding mean average precision
    mean_average_precison = sv.MeanAveragePrecision.from_tensors(
                predictions= pred_results,
                targets= gt_results,
            )
    # print(mean_average_precison.per_class_ap50_95)
    map_fcbk , map_zigzag = mean_average_precison.per_class_ap50_95.mean(axis = 1)
    df = df.copy(deep = True)
    df.loc['overall']['map50'] = mean_average_precison.map50
    df.loc['overall']['map75'] = mean_average_precison.per_class_ap50_95[:,5].mean()
    df.loc['overall']['map50-95'] = mean_average_precison.map50_95
    df.loc['FCBK']['map50'] = mean_average_precison.per_class_ap50_95[:,0][0]
    df.loc['FCBK']['map75'] = mean_average_precison.per_class_ap50_95[:,5][0]
    df.loc['FCBK']['map50-95'] = map_fcbk
    df.loc['Zigzag']['map50'] = mean_average_precison.per_class_ap50_95[:,0][1]
    df.loc['Zigzag']['map75'] = mean_average_precison.per_class_ap50_95[:,5][1]
    df.loc['Zigzag']['map50-95'] = map_zigzag
    print(f'Fold {fold}')
    print(df,end = '\n\n')

    a.append(df)

In [ ]:
print('Combined MAP of all folds ')
(a[0]+a[1]+a[2]+a[3])/num_folds
mean_and_std = {}
maps = ["map50", "map75", "map50-95"]
for map in maps:
    mean_and_std[map] = {}
    for i in range(2):
        if i==0:
            type = 'FCBK'
        else:
            type = 'Zigzag'
        values = []
        for j in range(4):
            values.append(a[j].loc[type][map])
        mean_and_std[map][type] = (np.mean(values), np.std(values))


print(mean_and_std)